In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/Dataset/brown_spot_Vs_non-brown_spot/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 4042 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/brown_spot_Vs_non-brown_spot/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 512 images belonging to 2 classes.


In [5]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [6]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [8]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [9]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[early])

Epoch 1/10
808/808 [==============================] - 2665s 3s/step - loss: 0.5538 - accuracy: 0.7674 - val_loss: 0.3312 - val_accuracy: 0.8490
Epoch 2/10
808/808 [==============================] - 2486s 3s/step - loss: 0.3410 - accuracy: 0.8477 - val_loss: 0.2860 - val_accuracy: 0.8725
Epoch 3/10
808/808 [==============================] - 3522s 4s/step - loss: 0.2824 - accuracy: 0.8813 - val_loss: 0.2378 - val_accuracy: 0.8902
Epoch 4/10
808/808 [==============================] - 3125s 4s/step - loss: 0.2373 - accuracy: 0.9108 - val_loss: 0.2383 - val_accuracy: 0.8922
Epoch 5/10
808/808 [==============================] - 3012s 4s/step - loss: 0.2276 - accuracy: 0.9051 - val_loss: 0.1787 - val_accuracy: 0.9157
Epoch 6/10
808/808 [==============================] - 2682s 3s/step - loss: 0.2009 - accuracy: 0.9215 - val_loss: 0.2349 - val_accuracy: 0.9255
Epoch 7/10
808/808 [==============================] - 2578s 3s/step - loss: 0.1923 - accuracy: 0.9257 - val_loss: 0.1985 - val_accuracy: